In [51]:
# IMPORT STATEMENTS
import pandas as pd
import csv

In [52]:
# IMPORT DATA
df = pd.read_csv('nba_player_game_logs.csv')
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x).rename(columns=lambda x: x.strip()) # remove whitespace

In [53]:
# GET PLAYOFF GAME VALUES 
playoff_start = '2022-04-16'
playoff_end = '2022-06-16'
playoff_values = df.loc[df['game_date'].between(playoff_start, playoff_end)] # create new Dataframe from playoff game values
playoff_teams = playoff_values['TEAM'].unique() # get each playoff team


In [54]:
# GET PLAYOFF PLAYERS 
# get all players who are on playoff teams and played at all in the playoffs
playoff_players = playoff_values[(playoff_values['MIN'] > 0) & (playoff_values['TEAM'].isin(playoff_teams))]
playoff_players_list = list(playoff_players.groupby(['PLAYER_ID', 'PLAYER_NAME']).groups) 

In [55]:
# GET REGULAR SEASON VALUES
regular_season_start = '2021-10-19'
regular_season_end = '2022-04-10'
regular_season_values = df.loc[df['game_date'].between(regular_season_start, regular_season_end)] # create new Dataframe from regular season game values

In [56]:
# get players who played on playoff teams but also played in regular season
regular_season_players = regular_season_values[(regular_season_values['MIN'] > 0) & (regular_season_values['TEAM'].isin(playoff_teams))]
regular_season_players_list = list(regular_season_players.groupby(['PLAYER_ID', 'PLAYER_NAME']).groups)

# compare sets of regular season and playoff players to see who only played in regular season 
set_difference = set(regular_season_players_list).difference(set(playoff_players_list))

In [57]:
# WRITE TO FILES

# write to csv files; could also convert list back to DataFrames and write to_csv 
with open('regular_season_only.csv', 'w', newline='') as new_file:

    csv_writer = csv.writer(new_file)
    csv_writer.writerow(['PLAYER_ID', 'PLAYER_NAME']) # column names
    csv_writer.writerows(set_difference)

with open('playoffs_only.csv', 'w', newline='') as new_file:

    # compare sets of regular season and playoff players to see who only played in playoffs 
    set_difference = set(playoff_players_list).difference(set(regular_season_players_list))
    csv_writer = csv.writer(new_file)
    csv_writer.writerow(['PLAYER_ID', 'PLAYER_NAME']) # column names
    csv_writer.writerows(set_difference)